In [1]:
!pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 3.9 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import shutil

In [4]:
!pip install transformers
!pip install --upgrade torch torchvision torchaudio
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 4.0 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2


In [5]:
!pip install fuzzywuzzy[speedup]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 48.7 MB/s eta 0:00:00


In [6]:
import os
import shutil
from docx import Document
from transformers import pipeline
from fuzzywuzzy import process

# Initialize NER pipeline
ner_pipeline = pipeline('ner', grouped_entities=True)

# Extract text from .docx file
def extract_text_from_docx(docx_path):
    doc = Document(docx_path)
    return "\n".join([para.text for para in doc.paragraphs])

# Extract name using NER pipeline
def extract_name(resume_text, ner_pipeline):
    entities = ner_pipeline(resume_text)
    print(f"NER Entities: {entities}")
    for ent in entities:
        if ent['entity_group'] == 'PER':
            return ent['word'].strip()
    return "Unknown_Name"

# Extract current job title using fuzzy matching
def extract_current_job_title(text):
    job_title = [
        # Software Development and Engineering
        'Software Engineer', 'Full Stack Developer', 'Frontend Developer', 'Backend Developer',
        'Sr. Software Developer', 'Lead Developer', 'Technical Lead', 'Tech Lead', 'Solution Architect',
        'Software Architect', 'Engineering Manager', 'Programmer', 'Embedded Systems Engineer', 'Agile', 'Scrum Master','Sr. Business & Implementation Analyst ','DevOps Engineer', 'Cloud Engineer', 'Systems Engineer', 'Cloud Architect','Business Analyst',

        "Senior Business Analyst",               "Lead Business Analyst",
        "Business Implementation Manager",       "Business Process Analyst",
        "Sr. Implementation Specialist",         "Business Systems Analyst",
        "Senior Implementation Consultant",      "Business Transformation Analyst",
        "Senior Solutions Analyst",              "Business Process Improvement Consultant",
        "Enterprise Business Analyst",           "Senior Business Process Consultant",
        "Business Operations Manager",           "Sr. Change Management Analyst",
        "Senior Business Strategy Consultant",   "IT Business Analyst",
        "Business Intelligence Analyst",         "Data Business Analyst",
        "Product Implementation Manager",        "Business Development Analyst",
        "Senior Project Analyst",                "Business Integration Analyst",
        "Senior Process Analyst",                "Strategic Implementation Manager", 'Sr. iOS Developer',
        "Business Architect",

        # Telecommunications and Networking
        'Telecommunications Engineer', 'Telecommunications Technician', 'Network Engineer', 'Network Administrator',
        'Wireless Engineer', 'RF Engineer', 'Network Architect', 'Telecom Analyst', 'Telecom Manager',
        'Telecom Project Manager', 'Telecom Consultant', 'Telecom Specialist', 'Telecommunications Manager',
        'Voice Network Engineer', 'Fiber Optic Technician', 'Satellite Communications Engineer', 'heterogeneous and complex IT environments',

        # Quality Assurance and Testing
        'QA Engineer', 'QA Tester', 'Quality Assurance Analyst', 'Quality Control Engineer', 'QA Automation Engineer',
        'Test Engineer', 'QA Lead', 'Manual Tester', 'Automation Tester', 'SDET (Software Development Engineer in Test)',
        'Performance Tester', 'Penetration Tester', 'Security Tester', 'Usability Tester',

        # Data Science, AI, and Analytics
        'Data Scientist', 'Data Analyst', 'Machine Learning Engineer', 'AI Engineer', 'Big Data Engineer',
        'Data Engineer', 'Data Architect', 'BI Analyst', 'Business Intelligence Developer', 'Data Visualization Engineer',

        # Management and Leadership Roles (Tech & Non-Tech)
        'Project Manager', 'Program Manager', 'Product Manager', 'IT Manager', 'Technical Project Manager',
        'Development Manager', 'Engineering Manager', 'Team Lead', 'Tech Lead', 'Delivery Manager', 'IT Operations Manager',
        'CTO (Chief Technology Officer)', 'Director of Engineering', 'Solutions Manager',
        'Digital Transformation Manager', 'Business Development Manager', 'Cloud Operations Manager',

        # DevOps and Infrastructure
        'DevOps Engineer', 'Cloud Engineer', 'Infrastructure Engineer', 'Platform Engineer', 'Systems Engineer',
        'Systems Administrator', 'Cloud Architect', 'Cloud Solutions Architect', 'SRE (Site Reliability Engineer)',
        'Linux Administrator', 'Windows Administrator', 'IT Support Engineer', 'Infrastructure Manager',

        # Security and Compliance
        'Cybersecurity Analyst', 'Security Engineer', 'Security Architect', 'Information Security Manager',
        'Compliance Manager', 'Penetration Tester', 'Vulnerability Assessor', 'Network Security Engineer',
        'Cloud Security Architect',

        # Additional Tech-Related Roles
        'Technical Consultant', 'IT Consultant', 'Solutions Consultant', 'ERP Consultant', 'CRM Consultant',
        'Technical Support Engineer', 'Helpdesk Support', 'Business Analyst', 'Systems Analyst', 'Technology Evangelist',
        "Java Developer",
        "Python Developer",
        "Full Stack Developer",
        "Frontend Developer",
        "Backend Developer",
        "Mobile App Developer",
        "iOS Developer",
        "Android Developer",
        "Node.js Developer",
        "Ruby on Rails Developer",
        "PHP Developer",
        "C# Developer",
        "Angular Developer",
        "React Developer",
        "Vue.js Developer",
        "Django Developer",
        "Spring Boot Developer",
        "DevOps Engineer",
        "Software Engineer",
        "Cloud Developer",

        # Additional QA and Testing-Related Roles
        'Agile Tester', 'Manual Testing', 'Automation Testing', 'Quality Assurance', 'Automation Testing of Web Applications',
        'Mainframe Environment Tester (Waterfall)', 'QA Tester', 'Web Developer', 'Mobile App Developer', 'Python Developer',
        'QA Lead', 'Software Testing & Quality Assurance', 'Manual Tester', 'Senior Network Engineer', 'Software Quality Assurance (SQA)','Network Architect','Android Developer','iOS Developer', 'SDE (Automation or Build or HA)','Java Developer','Java or J2EE Developer', 'Software Engineer', 'Sr. Software Developer','Senior Data Warehouse Architect or Tech Lead','SDE (Automation or Build or HA)',
        'Tech Lead', 'Senior Software Engineer','Team Lead',
    ]

    sentences = text.split('\n')
    for sentence in sentences:
        best_match = process.extractOne(sentence, job_title_keywords)
        if best_match and best_match[1] >= 85:
            return best_match[0]
    return "Job_Title_Not_Found"

# Save resume to the appropriate category
def save_resume_to_category(name, job_title, original_path):
    if name == "Unknown_Name":
        name = "unknown"
    if job_title == "Job_Title_Not_Found":
        job_title = "no_job"

    categories = {
    # 1. Software Development & Engineering
        'SoftwareEngg&Architects': [
       'Software Engineer', 'Full Stack Developer', 'Frontend Developer', 'Backend Developer',
        'Sr. Software Developer', 'Lead Developer', 'Technical Lead', 'Tech Lead', 'Solution Architect',
        'Software Architect', 'Engineering Manager', 'Programmer', 'Embedded Systems Engineer', 'Agile', 'Scrum Master','Sr. Business & Implementation Analyst ','DevOps Engineer', 'Cloud Engineer', 'Systems Engineer', 'Cloud Architect','Business Analyst',

        "Senior Business Analyst",               "Lead Business Analyst",
        "Business Implementation Manager",       "Business Process Analyst",
        "Sr. Implementation Specialist",         "Business Systems Analyst",
        "Senior Implementation Consultant",      "Business Transformation Analyst",
        "Senior Solutions Analyst",              "Business Process Improvement Consultant",
        "Enterprise Business Analyst",           "Senior Business Process Consultant",
        "Business Operations Manager",           "Sr. Change Management Analyst",
        "Senior Business Strategy Consultant",   "IT Business Analyst",
        "Business Intelligence Analyst",         "Data Business Analyst",
        "Product Implementation Manager",        "Business Development Analyst",
        "Senior Project Analyst",                "Business Integration Analyst",
        "Senior Process Analyst",                "Strategic Implementation Manager", 'Sr. iOS Developer',
        "Business Architect",


    # 2. Data Science, AI & Analytics
        'Data Science,AI&Analytics': [
        'Data Scientist', 'Data Analyst', 'Machine Learning Engineer', 'AI Engineer',
        'Big Data Engineer', 'Data Engineer', 'Data Architect', 'BI Analyst',
        'Business Intelligence Developer', 'Data Visualization Engineer',
        'Business Analyst', 'Sr. Business & Implementation Analyst', 'Senior Business Analyst',
        'Lead Business Analyst', 'Business Implementation Manager', 'Business Systems Analyst',
        'Senior Implementation Consultant', 'Business Process Analyst', 'Business Transformation Analyst',
        'Business Process Improvement Consultant', 'Enterprise Business Analyst', 'IT Business Analyst',
        'Data Business Analyst', 'Business Intelligence Analyst', 'Business Architect',
        'Business Development Analyst', 'Strategic Implementation Manager', 'Product Implementation Manager',
        'Business Integration Analyst', 'Senior Process Analyst',
    ],

    # 3. Quality Assurance & Testing
        'QA,QE,SDET': [
        'QA Engineer', 'QA Tester', 'Quality Assurance Analyst', 'Test Engineer',
        'QA Automation Engineer', 'Manual Tester', 'Automation Tester', 'SDET (Software Development Engineer in Test)',
        'Performance Tester', 'Security Tester', 'Penetration Tester', 'Usability Tester',
        'Agile Tester', 'Software Quality Assurance (SQA)','SDET', 'QA Lead', 'Senior Network Engineer',
        'QA Engineer', 'QA Tester', 'Quality Assurance Analyst', 'Quality Control Engineer', 'QA Automation Engineer',
        'Test Engineer', 'QA Lead', 'Manual Tester', 'Automation Tester', 'SDET (Software Development Engineer in Test)',
        'Performance Tester', 'Penetration Tester', 'Security Tester', 'Usability Tester',
    ],

    # 4. DevOps & Infrastructure
        'PM&BA': [
        'DevOps Engineer', 'Cloud Engineer', 'Infrastructure Engineer', 'Platform Engineer',
        'Systems Engineer', 'Systems Administrator', 'Cloud Architect', 'Cloud Solutions Architect',
        'Site Reliability Engineer (SRE)', 'Linux Administrator', 'Windows Administrator',
        'IT Support Engineer', 'Infrastructure Manager'
    ],

    # 5. Telecommunications & Networking
        'CRM&ERPS': [
        'Telecommunications Engineer', 'Network Engineer', 'Network Architect', 'RF Engineer',
        'Telecom Analyst', 'Voice Network Engineer', 'Fiber Optic Technician',
        'Satellite Communications Engineer', 'Wireless Engineer', 'Network Administrator',
        'Telecommunications Manager', 'Telecom Project Manager'
    ],

    }

    category_dir = 'Other'  # Default to "Other" if job_title doesn't match any category
    for category, titles in categories.items():
        if job_title in titles:
            category_dir = category
            break
    category_dir = 'Other'  # Default to "Other" if job_title doesn't match any category

    if job_title == "no_job":
      category_dir = 'Other'
    if name == 'unknown':
      category_dir = 'Other'
    else:
        for category, titles in categories.items():
          if job_title in titles:
            category_dir = category
            break




    base_dir = f'/content/drive/MyDrive/outputresumecatgory/{category_dir}'
    if not os.path.exists(base_dir):
        os.makedirs(base_dir)

    new_file_name = f"{name}_{job_title}.docx"
    new_file_path = os.path.join(base_dir, new_file_name)

    shutil.move(original_path, new_file_path)
    return new_file_path

# Main execution block
if __name__ == '__main__':
    uploaded_dir = r'/content/drive/MyDrive/test/'
    resume_paths = [os.path.join(uploaded_dir, file) for file in os.listdir(uploaded_dir) if file.endswith('.docx')]

    for resume_path in resume_paths:
        # Extract resume text
        text = extract_text_from_docx(resume_path)

        # Extract the name using the transformer-based NER pipeline
        name = extract_name(text, ner_pipeline)

        # Extract job title from resume content
        current_job_title = extract_current_job_title(text)

        # Rename and save the file
        new_file = save_resume_to_category(name, current_job_title, resume_path)
        print(f"Renamed and saved file as: {new_file}")


No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision 4c53496 (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/998 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/token_classification.py:170: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="simple"` instead.
  warnings.warn(
